In [10]:
import pandas as pd
import spacy
import pickle
from tqdm import tqdm
from gensim.corpora import Dictionary
from sklearn.utils import shuffle
from collections import defaultdict
import numpy as np

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('WikiQACorpus/WikiQA-train.tsv', sep='\t')

In [4]:
df.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0


In [6]:
# Loading only the answers, which r 1 as Labels.
# Thus, then the actual answers are received for training
df = df[df['Label'] == 1]
df.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
75,Q16,how much is 1 tablespoon of water,D16,Tablespoon,D16-0,This tablespoon has a capacity of about 15 mL.,1
83,Q16,how much is 1 tablespoon of water,D16,Tablespoon,D16-8,In the USA one tablespoon (measurement unit) i...,1
84,Q16,how much is 1 tablespoon of water,D16,Tablespoon,D16-9,In Australia one tablespoon (measurement unit)...,1
98,Q17,how much are the harry potter movies worth,D17,Harry Potter,D17-13,The series also originated much tie-in merchan...,1


In [7]:
len(df)

1039

In [41]:
inputt = df['Question']
target = df['Sentence']
len(inputt), len(target)

(1039, 1039)

In [33]:
wordlist_x = set()
wordlist_y = set()

In [42]:
for sentence in inputt:
    for word in sentence.strip().split():
        wordlist_x.add(word)

for sentence in target:
    for word in sentence.strip().split():
        wordlist_y.add(word)

In [43]:
len(wordlist_x), len(wordlist_y)

(2079, 8951)

In [47]:
vocabulary = wordlist_x.union(wordlist_y)
vocabulary.add(" ")
len(vocabulary)

10048

In [22]:
# nlp = spacy.load('en_core_web_sm')
# def process_text(input_string, return_string=False, stem=False):
#     text = nlp(u'' + input_string)
#     if stem == True:
#         text = [tok.lemma_ for tok in text if (tok.is_alpha and not tok.is_stop)]
#     else:
#         text = [tok.lower_ for tok in text if (tok.is_alpha and not tok.is_stop)]
#     if return_string == True:
#         return " ".join(text)
#     return text

In [23]:
# for i in tqdm(range(df.shape[0])):
#         wordlist_x.append(process_text(df['Question'].iloc[i]))
# for i in tqdm(range(df.shape[0])):
#         wordlist_y.append(process_text(df['Sentence'].iloc[i]))

100%|█████████████████████████████████████████████████████████| 1039/1039 [00:03<00:00, 270.41it/s]


In [48]:
token_indices = dict((c, i) for (c, i) in enumerate(vocabulary))
indices_token = dict((i, c) for (c, i) in enumerate(vocabulary))

In [49]:
indices_token[" "]

8985

In [50]:
# function to give out list of indices of each word in a given asentence
def getWordIndices(sentence):
    wordIndices = []
    for word in sentence.strip().split(" "):
        try:
            wordIndices.append(indices_token[word])
        except KeyError:
            wordIndices.append(indices_token[" "])
    return wordIndices

In [52]:
# both the trains of x & y are now indices of their respective words
X_train = []
for each in inputt:
    X_train.append(getWordIndices(each))

y_train = []
for each in target:
    y_train.append(getWordIndices(each))

In [54]:
X_train[0], y_train[0]

([379, 1227, 4569, 6865, 8235],
 [9952,
  4569,
  6491,
  784,
  894,
  6491,
  4107,
  4242,
  9935,
  3322,
  9020,
  894,
  4569,
  4676])

In [56]:
# We reversed each words in each sentence, in the reverse order
x_train = [x[::-1] for x in X_train]

In [58]:
x_train[0]    # the words, now in reversed order

[8235, 6865, 4569, 1227, 379]

In [55]:
from keras.preprocessing.sequence import pad_sequences

# We now padd both the questions and answers, with the index of " "
# Default of pad is that it is done as pre...that is this is done before the words...which are now in reversed order
X_train = pad_sequences(x_train, value=8985)
# But in case of y_train, it is done as post...bcoz we want this pad after the words...which are not reversed like in X_train
y_train = pad_sequences(y_train, value=8985, padding="post")

In [62]:
from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Reshape
from keras.layers.wrappers import TimeDistributed

In [63]:
seq2seq_model = Sequential()